In [1]:
import sys
import os
import re
#sys.path = [p for p in sys.path if p.find('/opt/apps/software/') == -1]
from glob import glob
from IPython.display import display, HTML
%matplotlib widget
from matplotlib import pyplot as plt

from utils.constants import UNIVARIATE_ARCHIVE_NAMES as ARCHIVE_NAMES
from utils.constants import UNIVARIATE_DATASET_NAMES as DATASET_NAMES
from utils.utils import read_all_datasets, transform_labels, create_directory, run_length_xps, generate_results_csv, plot_epochs_metric
from utils.data_loading import get_multiple_data, predict, shifted_zscore, zscore, print_metric, tf_rmse, tf_pmse, rmse
import utils
from classifiers import inception

import numpy as np
import pandas as pd
import sklearn

# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

def tf_pmse_DA(y_true, y_pred):
    return tf_pmse(y_true, y_pred, idx=0)

def tf_pmse_5HT(y_true, y_pred):
    return tf_pmse(y_true, y_pred, idx=1)

def tf_pmse_pH(y_true, y_pred):
    return tf_pmse(y_true, y_pred, idx=2)

def tf_pmse_NE(y_true, y_pred):
    return tf_pmse(y_true, y_pred, idx=3)

# import keras
# keras.backend.tensorflow_backend._get_available_gpus()

Instructions for updating:
non-resource variables are not supported in the long term


Using TensorFlow backend.


In [2]:
# data_prefix = '/mnt/nfs/proj/in-vitro/iterate/results_014/model_style_008/training-A-RBE-97Hz/'
data_prefix = '/mnt/nfs/proj/in-vitro/iterate/results_014/model_style_008/training-B-RBE025-97Hz/'
voltammograms = glob(data_prefix + '/*MMA*/voltammograms.mat')
probes = ['MMA003W01R04', 'MMA004W01R04', 'MMA013W01R04', 'MMA018W01R04', 'MMA019W01R04', 'MMA022W01R04', 'MMA023W01R04', 'MMA024W01R04', 'MMA025W01R04']
good_probes = probes

hold_probe = probes[1]
# hold_probe=None
print('hold probe :', hold_probe)

output_directory = f'/mnt/nfs/proj/in-vitro/Leonardo/inception/results/mm/{hold_probe}/'

if not (os.path.exists(output_directory)):
    os.makedirs(output_directory, exist_ok=True)

# val_probe=None
val_probe=probes[5]
print('validation probe :', val_probe)

print(f'Leaving out probe {hold_probe}', flush=True)
print(f'Validation probe {val_probe}', flush=True)
print(f'Loading data', flush=True)

filter_files = lambda x : x.find('500kohm') == -1
print('No 500ohms')

# filter_files = lambda x : x.find('500kohm') > -1
# print('ONLY 500ohms')

# normalize_data = minmax
# revert_data = lambda x: minmax(x, inverse=True)

normalize_data = shifted_zscore
revert_data = lambda x: shifted_zscore(x, inverse=True)

# normalize_data = zscore
# revert_data = lambda x: zscore(x, inverse=True)

# normalize_data = lambda x: x
# revert_data = lambda x: x

x_train, y_train, x_val, y_val, x_test, y_test = get_multiple_data(data_prefix,
                                                                    probes=good_probes,
                                                                    holdout=hold_probe,
                                                                    val_probe=val_probe,
                                                                    number_of_minibatches=20,
                                                                    normalize_data=normalize_data,
                                                                    filter_files=filter_files,
                                                                    nrecords_per_session=400)
print('Data loaded')

if len(x_train.shape) == 2:  # if univariate
    print('adding singleton')
    # add a dimension to make it multivariate with one dimension
    x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
    x_val = x_val.reshape((x_val.shape[0], x_val.shape[1], 1))
    x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

input_shape = x_train.shape[1:]
output_shape = y_train.shape[1]


hold probe : MMA025W01R04
validation probe : MMA003W01R04
Leaving out probe MMA025W01R04
Validation probe MMA003W01R04
Loading data
No 500ohms
number of voltammograms files 36
after filter 36
after removing bad probes 36
number of holdout files 4
(52800, 999)
(52800, 4)
validation probe: MMA003W01R04
number of validation files 4
(52800, 999)
(52800, 4)
number of train files 28
(369600, 999)
(369600, 4)
Data loaded
adding singleton


In [6]:
# Fit the model

classifier = inception.Regression_INCEPTION(output_directory, input_shape, output_shape, verbose=1, build=True, nb_epochs=100, 
                                            metrics=[tf_pmse_DA, tf_pmse_5HT, tf_pmse_pH, tf_pmse_NE])
# classifier = Regression_INCEPTION(output_directory, input_shape, output_shape, verbose=1, build=True, nb_epochs=100)

metrics = classifier.fit(x_train, y_train, x_val, y_val, plot_test_acc=True)

display(HTML(metrics.to_html()))


mini batch size: 64
Train on 369600 samples, validate on 52800 samples
Epoch 1/100
369600/369600 [==============================] - 336s 909us/step - loss: 1.0213 - tf_pmse_DA: 935.3340 - tf_pmse_5HT: 942.5300 - tf_pmse_pH: 0.2288 - tf_pmse_NE: 864.1198 - val_loss: 4.5814 - val_tf_pmse_DA: 866.5776 - val_tf_pmse_5HT: 1234.1121 - val_tf_pmse_pH: 1.1679 - val_tf_pmse_NE: 835.1944
Epoch 2/100
369600/369600 [==============================] - 324s 878us/step - loss: 0.4181 - tf_pmse_DA: 765.3237 - tf_pmse_5HT: 752.4347 - tf_pmse_pH: 0.1347 - tf_pmse_NE: 657.3793 - val_loss: 1.5835 - val_tf_pmse_DA: 882.1802 - val_tf_pmse_5HT: 1315.3749 - val_tf_pmse_pH: 0.3107 - val_tf_pmse_NE: 945.7693
Epoch 3/100
369600/369600 [==============================] - 325s 879us/step - loss: 0.2943 - tf_pmse_DA: 654.9942 - tf_pmse_5HT: 625.4911 - tf_pmse_pH: 0.1113 - tf_pmse_NE: 544.2216 - val_loss: 1.7133 - val_tf_pmse_DA: 1408.0268 - val_tf_pmse_5HT: 970.8600 - val_tf_pmse_pH: 0.2861 - val_tf_pmse_NE: 1117.259

,rmse_DA,rmse_5HT,rmse_pH,rmse_NE,duration
0,1265.930014,1245.923786,0.322998,1109.621645,32467.59564


In [7]:
metrics = classifier.predict(x_test, y_test, x_train, y_train, return_df_metrics=True)
display(HTML(metrics.to_html()))

,rmse_DA,rmse_5HT,rmse_pH,rmse_NE,duration
0,1162.706101,1138.727825,0.334436,1091.193107,0.0


In [8]:
print(classifier.output_directory)

# model_path = os.path.join(classifier.output_directory, 'best_model.hdf5')
model_path = os.path.join(classifier.output_directory, 'last_model.hdf5')
# model_path = os.path.join(classifier.output_directory, 'model_init.hdf5')

model = keras.models.load_model(model_path, custom_objects={"tf_pmse_DA": tf_pmse_DA, "tf_pmse_5HT": tf_pmse_5HT, "tf_pmse_pH": tf_pmse_pH, "tf_pmse_NE": tf_pmse_NE})

xt, yt = x_test, y_test
# xt, yt = x_val, y_val

yp = model.predict(xt, batch_size=64)

yp = np.apply_along_axis(revert_data, axis=1, arr=yp) 
yt = np.apply_along_axis(revert_data, axis=1, arr=yt) 

rmse4 = rmse(yt, yp)

metrics2 = pd.DataFrame(data=np.zeros((1, 5), dtype=np.float), index=[0], columns=['rmse_DA', 'rmse_5HT', 'rmse_pH', 'rmse_NE', 'duration'])
metrics2['rmse_DA'] = rmse4[0]
metrics2['rmse_5HT'] = rmse4[1]
metrics2['rmse_pH'] = rmse4[2]
metrics2['rmse_NE'] = rmse4[3]
metrics2['duration'] = 0.0
        
display(HTML(metrics2.to_html()))


/mnt/nfs/proj/in-vitro/Leonardo/inception/results/mm/MMA025W01R04/


,rmse_DA,rmse_5HT,rmse_pH,rmse_NE,duration
0,10449.314065,10449.408678,7.61867,34504.139931,0.0


In [52]:
# def plot_concentrations(y):
#     fig, axs = plt.subplots(2, 2, sharey=False, tight_layout=True)
#     for ip in range(0,4):
#         axs[np.unravel_index(ip, axs.shape)].hist(y[:,ip])
# plot_concentrations(y_test)
# plot_concentrations(y_pred)

def plot_compare_test_pred(yt, yp):
    fig, axs = plt.subplots(2, 4, sharey=False, tight_layout=True, figsize=(12, 5))
    for ip in range(0,8):
        axsidx = np.unravel_index(ip, axs.shape)
        if ip < 4:
            axs[axsidx].hist(yt[:,ip])
        else:
            axs[axsidx].hist(yp[:,ip-4])

plot_compare_test_pred(yt, yp)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …